In [2]:
import os
import gc
import sys
import warnings
from pathlib import Path

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose

In [54]:
df_names = ['playerBoxScores', 'teamBoxScores', 'playerTwitterFollowers', 'teamTwitterFollowers', 'awards', 'events',
           'games', 'standings', 'rosters', 'transactions']
path = "data/train_vs/"

train_dict = {}
for index in range(len(df_names)):
    train_dict[df_names[index]] = pd.read_pickle(os.path.join(path,df_names[index]) + "_train.pkl")

In [55]:
df_names = ['seasons', 'teams', 'players', 'awards']
path = "data/"

my_dict = {}
for index in range(len(df_names)):
    my_dict[df_names[index]] = pd.read_csv(os.path.join(path,df_names[index]) + ".csv")

In [56]:
target_v = pd.read_pickle('data/target_v/nextDayPlayerEngagement_train.pkl')

In [94]:
target_dict = {id: target_v[target_v['playerId']==id].drop(
    ['index'], axis = 1) for id in target_v['playerId'].unique()}    

In [95]:
target_dict[112526].head(3)

,engagementMetricsDate,playerId,target1,target2,target3,target4,date
969,2018-01-02,112526,0.055277,5.496109,0.025839,16.176471,20180101
3556,2018-01-03,112526,0.060625,3.252914,0.030486,8.541353,20180102
5737,2018-01-04,112526,0.029341,1.648352,0.032613,10.490112,20180103


## Dictionary for playerBoxScores

In [198]:
perform_stats = pd.DataFrame()
perform_stats = train_dict['playerBoxScores'][[
    'playerId', 'gamePk', 'gameDate', 'gameTimeUTC', 'teamId', 'date']]
perform_stats.head(3)

,playerId,gamePk,gameDate,gameTimeUTC,teamId,date
0,605131,529418,2018-03-29,2018-03-29T23:08:00Z,119,20180329
1,605480,529406,2018-03-29,2018-03-29T20:00:00Z,139,20180329
2,546318,529416,2018-03-29,2018-03-29T20:10:00Z,143,20180329


In [199]:
performance = train_dict['playerBoxScores']

In [200]:
perform_stats['FPCT'] = (performance['putOuts']+performance['assists']) / performance['chances']

# Not sure
perform_stats['Out'] = performance['putOuts'] + performance['groundOuts'] + performance['strikeOuts'] + performance['flyOuts']

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [201]:
perform_stats['AB'] = performance['atBats']

# Not sure
perform_stats['B2'] = performance['doubles']
perform_stats['B3'] = performance['triples']

perform_stats['SB'] = performance['stolenBases']
perform_stats['CS'] = performance['caughtStealing']
perform_stats['H'] = performance['hits']
perform_stats['HR'] = performance['homeRuns']
perform_stats['IBB'] = performance['intentionalWalks']
perform_stats['HBP'] = performance['hitByPitch']
perform_stats['LOB'] = performance['leftOnBase']
perform_stats['PA'] = performance['plateAppearances']
perform_stats['TB'] = performance['totalBases']
perform_stats['GIDP'] = performance['groundIntoDoublePlay']
perform_stats['RBI'] = performance['rbi']
perform_stats['BB'] = performance['baseOnBalls']
perform_stats['SO'] = performance['strikeOuts']

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer

In [202]:
perform_stats['SV_percnt'] = performance['saves']/performance['saveOpportunities']

perform_stats['HLD'] = performance['holds']

perform_stats['BS'] = performance['blownSaves']

perform_stats['IR'] = performance['inheritedRunners']

perform_stats['IP'] = performance['inningsPitched']

perform_stats['ERA'] = (9*performance['earnedRuns'])/perform_stats['IP']

perform_stats['WHIP'] = (perform_stats['BB'] + perform_stats['H'])/perform_stats['IP']

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

In [203]:
perform_stats['ISO'] = (perform_stats['B2']+2*perform_stats['B3']+3*perform_stats['HR'])/perform_stats['AB']

perform_stats['RC'] = perform_stats['TB']*(perform_stats['H']+perform_stats['BB'])/(
    perform_stats['AB']+perform_stats['BB'])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [205]:
playerBS_dict = {id: perform_stats[perform_stats[
    'playerId']==id] for id in target_v['playerId'].unique()}    

In [287]:
merge_1  = {}
for key, df in target_dict.items():
    df = target_dict[key].merge(playerBS_dict[key], on = ['date', 'playerId'], how = 'left')
    merge_1[key] = df

In [288]:
train_dict.keys()

dict_keys(['playerBoxScores', 'teamBoxScores', 'playerTwitterFollowers', 'teamTwitterFollowers', 'awards', 'events', 'games', 'standings', 'rosters', 'transactions'])

In [289]:
# train_dict['teamBoxScores'].columns

In [290]:
select_awards = train_dict['awards'][['playerId', 'date', 'awardId']]

In [291]:
awards_dict = {id: select_awards[select_awards[
    'playerId']==id] for id in target_v['playerId'].unique()}    

In [292]:
merge_2  = {}
for key, df in target_dict.items():
    df = merge_1[key].merge(awards_dict[key], on = ['date', 'playerId'], how = 'left')
    merge_2[key] = df

In [293]:
select_stand = train_dict['standings'][['gameDate', 'divisionId', 'teamId', 'divisionRank', 'leagueRank', 'wildCardRank', 
                                         'pct', 'homeWins', 'xWinLossPct', 'date']]

In [308]:
# train_dict['standings'].head()
merge_3  = {}
for key, df in target_dict.items():
    df = merge_2[key].merge(select_stand, on = ['date', 'teamId', 'gameDate'], how = 'left')
    merge_3[key] = df

In [309]:
train_dict['rosters'].head(3)

,playerId,gameDate,teamId,statusCode,status,index,date
0,400121,2018-01-01,116,A,Active,0,20180101
1,408045,2018-01-01,142,A,Active,0,20180101
2,425492,2018-01-01,120,A,Active,0,20180101


In [310]:
select_rosters = train_dict['rosters'].drop(['status', 'index'], axis = 1)

In [311]:
rosters_dict = {id: select_rosters[select_rosters['playerId']==id] for id in target_v['playerId'].unique()}

In [317]:
merge_4  = {}
for key, df in target_dict.items():
    df = merge_3[key].merge(rosters_dict[key], on = ['date', 'playerId', 'teamId', 'gameDate'], how = 'left')
    merge_4[key] = df

In [318]:
select_games = train_dict['games'][['gamePk', 'gameType', 'gameTimeUTC', 'codedGameState', 'gameNumber',
                                   'homeId', 'homeWinPct', 'homeScore',]]

In [319]:
merge_5  = {}
for key, df in target_dict.items():
    df = merge_4[key].merge(select_games, on = ['gameTimeUTC', 'gamePk'], how = 'left')
    merge_5[key] = df

In [320]:
merge_5[112526].columns

Index(['engagementMetricsDate', 'playerId', 'target1', 'target2', 'target3',
       'target4', 'date', 'gamePk', 'gameDate', 'gameTimeUTC', 'teamId',
       'FPCT', 'Out', 'AB', 'B2', 'B3', 'SB', 'CS', 'H', 'HR', 'IBB', 'HBP',
       'LOB', 'PA', 'TB', 'GIDP', 'RBI', 'BB', 'SO', 'SV_percnt', 'HLD', 'BS',
       'IR', 'IP', 'ERA', 'WHIP', 'ISO', 'RC', 'awardId', 'divisionId',
       'divisionRank', 'leagueRank', 'wildCardRank', 'pct', 'homeWins',
       'xWinLossPct', 'statusCode', 'gameType', 'codedGameState', 'gameNumber',
       'homeId', 'homeWinPct', 'homeScore'],
      dtype='object')

In [316]:
train_dict['events'].columns

Index(['gamePk', 'gameDate', 'gameTimeUTC', 'season', 'gameType', 'playId',
       'eventId', 'inning', 'halfInning', 'homeScore', 'awayScore',
       'menOnBase', 'atBatIndex', 'atBatDesc', 'atBatEvent', 'hasOut',
       'pitcherTeamId', 'isPitcherHome', 'pitcherTeam', 'hitterTeamId',
       'hitterTeam', 'pitcherId', 'pitcherName', 'isStarter', 'pitcherHand',
       'hitterId', 'hitterName', 'batSide', 'pitchNumber', 'balls', 'strikes',
       'isGB', 'isLD', 'isFB', 'isPU', 'launchSpeed', 'launchAngle',
       'totalDistance', 'event', 'description', 'rbi', 'pitchType', 'call',
       'outs', 'inPlay', 'isPaOver', 'startSpeed', 'endSpeed', 'nastyFactor',
       'breakAngle', 'breakLength', 'breakY', 'spinRate', 'spinDirection',
       'pX', 'pZ', 'aX', 'aY', 'aZ', 'pfxX', 'pfxZ', 'vX0', 'vY0', 'vZ0', 'x',
       'y', 'x0', 'y0', 'z0', 'type', 'zone', 'index', 'date'],
      dtype='object')

In [321]:
select_events = train_dict['events'][['gamePk', 'gameDate', 'gameTimeUTC','gameType', 'eventId', 
                                      'inning', 'halfInning', 'homeScore', 'hitterTeamId', 
                                     'hitterTeamId', 'pitchType', 'pitchType']]

In [ ]:
merge_6  = {}
for key, df in target_dict.items():
    df = merge_5[key].merge(select_events, on = ['gamePk', 'gameDate', 'gameTimeUTC','gameType'], how = 'left')
    merge_6[key] = df

In [ ]:
merge_6[112526].shape